<a href="https://colab.research.google.com/github/Camipinto/Laboratorio-de-Programacion-Cientifica-para-Ciencia-de-Datos/blob/main/Lab08/Lab08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Laboratorio 8: ¿Superhéroe o Villano? 🦸</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Matías Rojas y Mauricio Araneda
- Auxiliar: Ignacio Meza D.
- Ayudante: Rodrigo Guerra

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: **Camila Pinto L.**
- Nombre de alumno 2: **Camilo Ramírez C.**


### **Link de repositorio de GitHub:** https://github.com/Camilormz/MDS7202-labs

## Temas a tratar

- Codificación de texto usando Bag of Words.
- Búsqueda del modelo óptimo de clasificación usando `GridSearch`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- **Ausentes** deberán realizar la actividad solos. 
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Obtener caracteristicas a partir de texto usando `CountVectorizer`.
- Fijar un pipeline con un modelo base que luego se irá optimizando.
- Comprender como realizar una búsqueda de grilla sobre un conjunto de clasificadores e hiperparámetros usando `GridSearch`.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

#Importamos librerias utiles 😸

In [ ]:
# Librería Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split 

# Pre-procesamiento
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import CountVectorizer

# Clasifación
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

# Librería para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Proyecciones en baja dimensionalidad: UMAP
!pip install umap-learn

# Librería para NLP
!pip install nltk
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize  
from nltk.stem import PorterStemmer
nltk.download('stopwords')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 5.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 5.5.0
    Uninstalling plotly-5.5.0:
      Successfully uninstalled plotly-5.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 88 kB 3.4 MB/s 
     |████████████████████████████████| 1.1 MB 10.2 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82830 sha256=5467df2a0dc01e3a6f56d5a7ef6660e9250a01005e3d80517de5d4cae82a2a55
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55509 sha256=5b9db6f194c95bbc7d88ccfc482c1358d6e8882f4bba29c7a0e1064aa9367ac9
  Stored in directory: /root/.cache/pip/wheels/19/bc/

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# 1. ¿Quien es Bat Cow?

<p align="center">
  <img src="https://i.imgur.com/D9f1RHy.jpg" width="350">
</p>

En vez de estar desarrollando las evaluaciones correspondientes a su curso, su profesor de catedra y su auxiliar discuten acerca la alineación (héroe o villano) del personaje de ficción Bat-Cow. 

El cuerpo docente, no logra ponerse de acuerdo si el personaje es bueno, neutral o malo: el auxiliar plantea que Bat-cow posee una siniestra mirada, intrigante pero común característica de los personajes malvados. 
Por otra parte, extendiendo las ideas de Rousseau, el profesor plantea que tal como los humanos no nacen malos, no existe motivo por el cual una vaca con superpoderes deba serlo.

Sin embargo, ambos concuerdan que es difícil estimar la alineación solo usando los atributos físicos, por lo que creen el análisis debe ser complementado aún más antes de comunicarle los resultados a su estudiantado. Buscando más información, ambos sujetos se percatan de la existencia de un excelente antecedente para estimar la alineación: la historia personal de cada superhéroe o villano.

Es por esto le solicitan que construya y optimice un clasificador basado en texto el cual analice la alineación de cada personaje basado en su historia personal.

Para este laboratorio deben trabajar con los datos `df_comics.csv` y `comics_no_label.csv` subidos a u-cursos. El primero es un conjunto de datos que les servirá para entrenar un modelo de clasificación, mientras que el segundo es un dataset con personajes de ficción no etiquetados a predecir (sí, aquí está la misteriosa Batcow).

Para comenzar cargue los dataset señalados y visualice a través de un head los atributos que poseen cada uno de los dataset.


In [ ]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
#     from google.colab import drive
#     drive.mount("/content/drive")
#     path = 'Dirección donde tiene los archivos en el Drive'
# except: 
print('Ignorando conexión drive-colab')

Ignorando conexión drive-colab


In [ ]:
%%capture
# Descarga directa de los datos desde Internet
!wget -O df_comics.csv https://drive.google.com/u/0/uc?id=1AxfyjplbiUJngbI7ndcK3vl4m2bdgxMc&export=download
!wget -O comics_no_label.csv https://drive.google.com/u/0/uc?id=1mJC6VCptQGdsb44F4RkE8t6UI1jomO87&export=download

In [ ]:
from pandas.core.frame import DataFrame as PandasDF

df_comics: PandasDF = pd.read_csv('df_comics.csv')
df_comics_no_label: PandasDF = pd.read_csv('comics_no_label.csv')
# Eliminar ejemplos sin historia
df_comics = df_comics.dropna(subset=['history_text'])

In [ ]:
# Queda a labor de su equipo hacer el análisis exploratorio
df_comics.head()

,Unnamed: 0,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,2,A-Bomb,Richard Milhouse Jones,Richard Milhouse Jones,20,"Richard ""Rick"" Jones was orphaned at a young ...","On rare occasions, and through unusual circu...",80,100,80,...,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,3,Aa,Aa,NaN,12,Aa is one of the more passive members of the P...,NaN,80,50,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Aaron Cash,Aaron Cash,Aaron Cash,5,Aaron Cash is the head of security at Arkham A...,NaN,80,10,25,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Aayla Secura,Aayla Secura,NaN,8,ayla Secura was a Rutian Twi'lek Jedi Knight (...,NaN,90,40,45,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## 1.1 Obtención de Features y Bag of Words

<p align="center">
  <img src="https://media0.giphy.com/media/eIUpSyzwGp0YhAMTKr/200.gif" width="300">
</p>

Primero que todo, deben obtener un vector de características del atributo `history_text`, utilizando `Bag of Words`. En este atributo se presenta una breve descripción de la historia de cada uno de los personajes de ficción presentes en el dataset. 

Pero... antes de empezar, ¿Que es `Bag of Words`?...

`Bag of Words` es un modelo de conteo utilizado en Procesamiento de Lenguaje Natural (NLP) que tiene como objetivo generar una representación vectorial (vector de características en nuestro cas) para cada documento a través del conteo de las palabras que contienen. 

La siguiente figura muestra un ejemplo de `Bag of Words` en acción:

<p align="center">
  <img src="https://user.oc-static.com/upload/2020/10/23/16034397439042_surfin%20bird%20bow.png" width="500">
</p>

Como pueden ver, el modelo de `Bag of Words` no resulta tan complicado, ¿pero cómo lo aplicamos en python?. 

Como podrán darse cuenta del ejemplo anterior, para facilitar el conteo será necesario transformar cada uno de los documentos en vectores, donde cada una de las posiciones posee un carácter. Este proceso es conocido como **tokenización** y lo podemos realizar de la siguiente forma:

In [ ]:
from typing import List

nltk.download('punkt')

docs: List[str] = ['The teacher rocks like a good rock & roll',
                   'the rock is the best actor in the world']

docs_tokenizados: List[List[str]] = [word_tokenize(doc) for doc in docs]
docs_tokenizados

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[['The', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world']]

Podemos mejorar un poco más el proceso de tokenización agregando 

- Stemming:  Definimos Stemming como un algoritmo basado en reglas que transforma las palabras a una forma general. Un ejemplo de stemming, es el siguiente:
- Eliminación de Stopwords: Eliminación de palabras muy frecuentes que entorpecen la clasificación (por ejemplo, el, la los, la, etc...)

<p align="center">
  <img src="https://devopedia.org/images/article/218/8583.1569386710.png" width="300">
</p>


In [ ]:
# Definimos algunas stopword que queremos que sean eliminadas
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

stop_words: List[str] = stopwords.words('english')

# Definimos un tokenizador con Stemming
class StemmerTokenizer:
    """
    Clase de tokenizador con Stemming

    Attributes
    ----------
    ps
        Objeto que representa el stemmer de palabras del algoritmo de stemming
        de Porter 
    """
    def __init__(self):
        """
        Constructor de clase
        """
        self.ps: PorterStemmer = PorterStemmer()

    def __call__(self, doc) -> List[str]:
        """
        Llamada sobre el stemizador

        Parameters
        ----------
        doc: str
            Docuemento a tokenizar con stemming
        """
        doc_tok: List[str] = word_tokenize(doc)
        doc_tok = [t.lower() for t in doc_tok if t not in stop_words]
        return [self.ps.stem(t) for t in doc_tok]

# Inicializamos tokenizador
tokenizador: StemmerTokenizer = StemmerTokenizer()

# Creamos algunos documentos
docs: List[str] = ['The teacher rocks like a good rock & roll',
                   'the rock is the best actor in the world',
                   'New York is a beautiful city']

# Obtenemos el token del primer documento
[tokenizador(doc) for doc in docs]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['the', 'teacher', 'rock', 'like', 'good', 'rock', '&', 'roll'],
 ['rock', 'best', 'actor', 'world'],
 ['new', 'york', 'beauti', 'citi']]

In [ ]:
# Comparación con el caso anterior
docs_tokenizados = [word_tokenize(doc.lower()) for doc in docs]
docs_tokenizados

[['the', 'teacher', 'rocks', 'like', 'a', 'good', 'rock', '&', 'roll'],
 ['the', 'rock', 'is', 'the', 'best', 'actor', 'in', 'the', 'world'],
 ['new', 'york', 'is', 'a', 'beautiful', 'city']]

#### Al Estilo Scikit

Scikit implementa `bag of words` a través de la clase `CountVectorizer()` la cual contiene muchas opciones para mejorar la tokenización.

In [ ]:
from scipy.sparse.csr import csr_matrix

bow: CountVectorizer = CountVectorizer(tokenizer=StemmerTokenizer())
df: csr_matrix = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,actor,beauti,best,citi,good,like,new,rock,roll,teacher,world,york
0,1,0,0,0,0,1,1,0,2,1,1,0,0
1,0,1,0,1,0,0,0,0,1,0,0,1,0
2,0,0,1,0,1,0,0,1,0,0,0,0,1


Una de las cosas más interesantes que provee son el use de n-gramas, los cuales, en palabras simples, son conjuntos de n-palabras que se concatenan entre si y que se consideran como tokens separados. 

Pensemos en `Nueva York`. Cuando se tokeniza Nueva York, se generan dos tokens independientes que a simple vista no tienen relación: `Nueva` `York`.
Al usar n-gramas (en un rango min=1,max=2) , generamos tanto `Nueva` y `York` como también `Nueva York` como un token independiente.

In [ ]:
bow: CountVectorizer = CountVectorizer(
    tokenizer= StemmerTokenizer(), ngram_range=(1, 2)
)
df: csr_matrix = bow.fit_transform(docs)

pd.DataFrame(df.toarray(), columns=bow.get_feature_names_out())

,&,& roll,actor,actor world,beauti,beauti citi,best,best actor,citi,good,...,rock,rock &,rock best,rock like,roll,teacher,teacher rock,world,york,york beauti
0,1,1,0,0,0,0,0,0,0,1,...,2,1,0,1,1,1,1,0,0,0
1,0,0,1,1,0,0,1,1,0,0,...,1,0,1,0,0,0,0,1,0,0
2,0,0,0,0,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1


De los resultados, podemos ver que generamos vectores de conteo para cada una de las palabras que conforman el corpus.  Un punto extra que se agrega en esta obtención de frecuencias son los bigramas, que básicamente son el conjunto de palabras de tamaño de aparecen juntas en el texto.

## Codificando los Super{heroes, villanos}  [0.5 Puntos]

<p align="center">
  <img src="https://c.tenor.com/LkQzw7k5DV4AAAAd/anime-hacking.gif" width="300">
</p>

Conociendo ahora que es el proceso de `bag of words`, aplique este modelo de obtención de caracteristicas de la siguiente forma en un pipeline:

- Utilice el tokenizador entregado.
- Obtenga caracteristicas de los unigramas y bigramas del texto (tal como el ejemplo).

```python
bog = CountVectorizer(tokenizer= StemmerTokenizer(),`
                      ngram_range=(1,2) # Este punto es opcional y es para generar bigramas
                      )
```

Finalmente, aplique `MinMaxScaler()` sobre `atributos_de_interes` y concatene el valor obtenido con el matriz de caracteristicas obtenidas con bag of words.

```python
atributos_de_interes = ['intelligence_score', 'strength_score', 'speed_score', 'durability_score', 'power_score', 'combat_score']
```

No es necesario que obtenga un dataframe en concreto con las características solicitadas. Se le recomienda generar un `ColumnTransformer()` para aplicar las transformaciones solicitadas en un pipeline.

**To-Do:**
- [X] Obtener a traves de Bag of Words (`CountVectorizer`) caracteristicas del resumen de historia de cada personaje.
- [X] Aplicar `MinMaxScaler` sobre los atributos de interes.

**Respuesta:**

In [ ]:
bog: CountVectorizer = CountVectorizer(
    tokenizer=StemmerTokenizer(), ngram_range=(1, 2)
)

atributos_de_interes: List[str] = [
    "intelligence_score", "strength_score", "speed_score", "durability_score",
    "power_score", "combat_score"
]

column_transformer: ColumnTransformer = ColumnTransformer(transformers=[
    ("MinMax", MinMaxScaler(), atributos_de_interes),
    ("BagOfWords", bog, "history_text")
])

column_transformer.fit_transform(df_comics)

<1285x333931 sparse matrix of type '<class 'numpy.float64'>'
	with 794027 stored elements in Compressed Sparse Row format>

In [ ]:
applied_transform: csr_matrix = column_transformer.fit_transform(df_comics)

## 1.2 Diseño de Baseline y  Primer Entrenamiento  [1 Puntos]


<p align="center">
  <img src="https://pa1.narvii.com/6374/9eaec1b7bf9157334151452a669516f9a78b954c_hq.gif" width="300">
</p>



Genere un Pipeline con las caracteristicas solicitadas en la sección 1.1, un selector de mejores features `SelectPercentile` con métrica `f_classif` y percentile=90 y un clasificador `MultinomialNB()` por defecto.

Luego, separe el conjunto de datos en un conjunto de entrenamiento y prueba, donde las etiquetas estará dado por el atributo `alignment`. 

Entrene el modelo y reporte el desempeño con un `classification_report`. ¿ Nos recomendaría predecir la alineación de BatCow con este clasificador?.

Finalmente, compare el modelo entrenado con un modelo Dummy estratificado y responda: ¿El clasificador entrenado es mejor que el dummy que entrega respuestas al azar?

**To-do:**
- [X] Realizar un pipeline con las caracteristicas solicitadas en 1.1, ejecutar holdout y aplicar un clasificador `MultinomialNB()`.
- [X] Entrenar el pipeline, calcular el `classification_report` asociado y comentar los resultados.
- [X] Entrenar un `DummyClassifier` con estrategia `statified`, calcular el `classification_report` asociado y comentar que implican los scores obtenidos en comparación con los resultados del baseline.

**Respuesta:**

In [ ]:
from pandas.core.series import Series as PandasSeries
from sklearn.dummy import DummyClassifier

# Generación del pipeline
nb_pipe: Pipeline = Pipeline(steps=[
    ("preprocessing", column_transformer),
    ("feature_selection", SelectPercentile(f_classif, percentile=90)),
    ("classifier", MultinomialNB())
])

# Holdout de los datos
labels: PandasSeries = df_comics["alignment"]
X_train, X_test, y_train, y_test = train_test_split(
    df_comics, labels, test_size=0.3, shuffle=True, stratify=labels,
    random_state=1234
)

# Entrenamiento del pipeline
nb_pipe.fit(X_train, y_train)

# Evaluación del entrenamiento
print("----- Classification report for MultinomialNB classifier -----")
print(classification_report(y_test, nb_pipe.predict(X_test)))

# Generación del pipeline dummy, entrenamiento y evaluación del mismo
dummy_pipe: Pipeline = Pipeline(steps=[
    ("preprocessing", column_transformer),
    ("feature_selection", SelectPercentile(f_classif, percentile=90)),
    ("classifier", DummyClassifier(strategy="stratified"))
])
dummy_pipe.fit(X_train, y_train)
print("----- Classification report for dummy classifier -----")
print(classification_report(y_test, dummy_pipe.predict(X_test)))

----- Classification report for MultinomialNB classifier -----
              precision    recall  f1-score   support

         Bad       0.65      0.09      0.15       129
        Good       0.59      0.98      0.74       223
     Neutral       1.00      0.06      0.11        34

    accuracy                           0.60       386
   macro avg       0.75      0.37      0.33       386
weighted avg       0.65      0.60      0.49       386

----- Classification report for dummy classifier -----
              precision    recall  f1-score   support

         Bad       0.33      0.33      0.33       129
        Good       0.59      0.60      0.59       223
     Neutral       0.10      0.09      0.09        34

    accuracy                           0.46       386
   macro avg       0.34      0.34      0.34       386
weighted avg       0.46      0.46      0.46       386



> Respecto al modelo *Naive Bayes* multinomial por defecto, este tiene métricas que no destacan, particularmente un *accuracy* de 60% y *f1-score* promedio de 0.33 (macro) y 0.49 ponderado; este no es un modelo que se recomendaría para predecir la alneación de *BatCow* sino que se recomendaría iterar sobre clasificadores; sobre todo considerando que el *Naive Bayes* multinomial asume una distribución multinomial (categórica) de los atributos, lo cual ignora los atributos numéricos que no fueron construidos a partir del *bag of words*.

> Luego, comparando el modelo NB con respecto al clasificador *dummy*, se puede ver que este último tiene peores desemepños en general; particularmente su *accuracy* es de 44%, menor al 60% de *accuracy* del modelo *naive bayes*; esto da a entender que *naive bayes* sería el *baseline*, al ser precísamente mejor que el clasificador *dummy*.

## 1.3 Busqueda del Mejor Modelo con Grid Search [4 Puntos]

<p align="center">
  <img src="https://media1.tenor.com/images/70fdfeea52a8e2e4505498c230a0d2f9/tenor.gif?itemid=5134219" width="250">
</p>

No conformes con el rendimiento obtenido en la sección 1.2, el cuerpo docente les pide que realicen un **`HalvingGridSearchCV`** con diferentes parámetros para mejorar el rendimiento de la clasificación. Para esto, se le solicita que defina:

- Tres clasificadores distintos en donde varie sus parámetros. Considere usar modelos clásicos como también los basados en ensamblaje.
- Modificar `n-gram` range del `CountVectorizer` probando `(1,1), (1,2) y (1,3)`. Examinar también los otros parámetros de CountVectorizer como por ejemplo `max_df`, `min_df`, etc... ([Documentación aquí](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html))
- Seleccionar las columnas que contribuyen con la mayor información para la clasificación con `SelectPercentile` en los percentiles `[20, 40, 60, 80]` (puede usar la métrica que usted quiera).
- Reporte la mejor combinación encontrada y justifique por qué cree que es la mejor según el clasificador usado, la cantidad de columnas seleccionadas y los parámetros de CountVectorizer seleccionados por GridSearch.

A continuación, un ejemplo de parametros para GridSearch para una búsqueda de 3 clasificadores distintos:

```python
params = [
       # clasificador 1 + hiperparámetros
       {'clf': classificator1(),
        'clf__penalty': ['ovr'],
       # clasificador 1 + hiperparámetros    
       {'clf': classificator2(),
        'clf__n_estimators': [200]},
       # clasificador 1 + hiperparámetros
       {'clf': classificator3(),
        ...
       }
       ]
```

**Nota 1**: Puede ver los parámetros modificables aplicando el método get_params() sobre su pipeline. Ver la clase de GridSearch para mayor información sobre la sintáxis de las grillas.

**Nota 2**: Recuerde inicializar los clasificadores con un random state definido.

**Nota 3**: Puede usar en `HalvingGridSearchCV` el parámetro `verbose=10` para ver que GridSearch le indique el estado de su ejecución.

**Nota 3:** El GridSearch puede tomar tiempos de búsqueda exorbitantes, por lo que se le recomienda no agrandar mucho el espacio de búsqueda, dejar corriendo el código y tomarse un tecito.

**Respuesta:**

In [ ]:
from sklearn.experimental import enable_halving_search_cv

from sklearn.model_selection import HalvingGridSearchCV
from typing import Dict, Any

common_parameters: Dict[str, Any] = {
    "preprocessing": [column_transformer],
    "preprocessing__BagOfWords__ngram_range": [(1, 1), (1, 2), (1, 3)],
    "preprocessing__BagOfWords__max_df": [0.8, 1.0],
    "preprocessing__BagOfWords__min_df": [1, 2],
    "feature_selection__percentile": [20, 40, 60, 80]
}
nb_parameters: Dict[str, Any] = {
    "classifier": [MultinomialNB()], "classifier__alpha": [0.0, 0.5, 1.0]
}
rf_parameters: Dict[str, Any] = {
    "classifier": [RandomForestClassifier(random_state=1234)],
    "classifier__n_estimators": [50, 100, 200],
    "classifier__criterion": ["gini", "entropy"]
}
svc_parameters: Dict[str, Any] = {
    "classifier": [SVC(random_state=1234)], "classifier__C": [0.5, 1.0],
    "classifier__kernel": ["linear", "rbf"]
}
parameter_grid: List[Dict[str, Any]] = [
    {**common_parameters, **nb_parameters},
    {**common_parameters, **rf_parameters},
    {**common_parameters, **svc_parameters}
]
hsv: HalvingGridSearchCV = HalvingGridSearchCV(
    estimator=dummy_pipe, param_grid=parameter_grid, cv=3, n_jobs=-1,
    verbose=10
)
hsv.fit(X_train, y_train)

n_iterations: 4
n_required_iterations: 6
n_possible_iterations: 4
min_resources_: 18
max_resources_: 899
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 624
n_resources: 18
Fitting 3 folds for each of 624 candidates, totalling 1872 fits
----------
iter: 1
n_candidates: 208
n_resources: 54
Fitting 3 folds for each of 208 candidates, totalling 624 fits
----------
iter: 2
n_candidates: 70
n_resources: 162
Fitting 3 folds for each of 70 candidates, totalling 210 fits
----------
iter: 3
n_candidates: 24
n_resources: 486
Fitting 3 folds for each of 24 candidates, totalling 72 fits


HalvingGridSearchCV(cv=3,
                    estimator=Pipeline(steps=[('preprocessing',
                                               ColumnTransformer(transformers=[('MinMax',
                                                                                MinMaxScaler(),
                                                                                ['intelligence_score',
                                                                                 'strength_score',
                                                                                 'speed_score',
                                                                                 'durability_score',
                                                                                 'power_score',
                                                                                 'combat_score']),
                                                                               ('BagOfWords',
                                    

In [ ]:
# Se visualizan los parámetros del mejor pipeline encontrado
hsv.best_params_

{'classifier': RandomForestClassifier(criterion='entropy', n_estimators=50, random_state=1234),
 'classifier__criterion': 'entropy',
 'classifier__n_estimators': 50,
 'feature_selection__percentile': 80,
 'preprocessing': ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                  ['intelligence_score', 'strength_score',
                                   'speed_score', 'durability_score',
                                   'power_score', 'combat_score']),
                                 ('BagOfWords',
                                  CountVectorizer(min_df=2, ngram_range=(1, 2),
                                                  tokenizer=<__main__.StemmerTokenizer object at 0x7f3b613bb750>),
                                  'history_text')]),
 'preprocessing__BagOfWords__max_df': 1.0,
 'preprocessing__BagOfWords__min_df': 2,
 'preprocessing__BagOfWords__ngram_range': (1, 2)}

> Como se puede ver, se consideraron 80% de las columnas, además que para el `CountVectorizer` se descartaron los *n-grams* que tenían una única aparición en el texto. El clasificador encontrado fue un *random forest* que opimizó sobre la entropía.

> Respecto a los parámetros de pre-procesamiento, se cree que estos son mejores debido a que descartan columnas poco informativas y que en general hacían a los algoritmos de clasificación demorar en converger al incorporar componentes ruidosas.

> Respecto al clasificador, si bien se podría esperar que el *Naive-Bayes* Multinomial tuviera buen desemepeño al estar procesando lenguaje natural, los *Random Forest* son útiles para clasificar en base a las características numéricas, lo cual se estima que puede significar una mejora del modelo.

## 1.4 Predicción del datos sin etiquetado  [0.5 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/DolotxUUYAAbg7f.jpg" width="350">
</p>


LLego el momento de predecir 
`Vergil`, `Gorilla Girl` y `Batcow`


**Nota:** Recuerde que pueden existir campos vacios en `history_text`, por lo que se les recomienda borrar los nan.

**Respuesta:**

In [ ]:
subjects: PandasDF = df_comics_no_label[
    df_comics_no_label["name"].isin(["Vergil", "Gorilla Girl", "Batcow"])
].dropna(subset=['history_text']).drop_duplicates()

In [ ]:
hsv.predict(subjects)

array(['Good', 'Good', 'Good'], dtype=object)

Con estos resultados, es posible ver que efectivamente `Batcow` es un heroe.

<p align="center">
  <img src="https://media1.tenor.com/images/fb5bf7cc5a4acb91b4177672886a88ba/tenor.gif?itemid=5591338">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>